In [8]:
# pip install opencv-python

import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd

In [12]:
def get_table(sims):
    """
    This function takes the output produced by either the compute_similarities \ 
    or compute_similarities_testsets function, and returns a pandas dataframe/table \
    and also saves it in excel.
    """
    
    data = {}
    rows = []

    for key, value in sims.items():
        if key[0] not in data:
            data[key[0]] = []
        if key[1] not in rows:
            rows.append(key[1])
        data[key[0]].append(value)
        
    data = {key[:key.rfind(".")]:value for key, value in data.items()}
    rows = [row[:row.rfind(".")] for row in rows]
        
    df = pd.DataFrame(data, index=rows)
    #df.to_excel('output.xlsx')
    return df.T
    


<>:2: SyntaxWarning: invalid escape sequence '\ '
<>:2: SyntaxWarning: invalid escape sequence '\ '
C:\Users\used\AppData\Local\Temp\ipykernel_11468\1270361527.py:2: SyntaxWarning: invalid escape sequence '\ '
  """


In [13]:
def extract_orb_features(image_path):
    # Load image
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError(f"Image at path {image_path} could not be loaded.")
    
    # Initialize ORB detector
    orb = cv2.ORB_create()
    
    # Find keypoints and descriptors
    kp, des = orb.detectAndCompute(img, None)
    return kp, des

def match_descriptors(des1, des2):
    matches = []
    for i, d1 in enumerate(des1):
        distances = np.linalg.norm(des2 - d1, axis=1)
        min_idx = np.argmin(distances)
        matches.append((i, min_idx, distances[min_idx]))
    return matches

def ratio_test(matches, ratio_threshold=0.75):
    good_matches = []
    for i, (query_idx, train_idx, distance) in enumerate(matches):
        if i < len(matches) - 1:
            next_distance = matches[i + 1][2]
            if distance < ratio_threshold * next_distance:
                good_matches.append((query_idx, train_idx))
    return good_matches

def compute_orb_similarity(image_path1, image_path2):
    # Extract ORB features
    kp1, des1 = extract_orb_features(image_path1)
    kp2, des2 = extract_orb_features(image_path2)
    
    if des1 is None or des2 is None:
        return 0.0

    # Match descriptors manually
    matches = match_descriptors(des1, des2)
    
    # Apply ratio test
    good_matches = ratio_test(matches)
    
    if len(good_matches) < 4:
        return 0.0
    
    src_pts = np.float32([kp1[i].pt for i, _ in good_matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[i].pt for _, i in good_matches]).reshape(-1, 1, 2)
    
    # Use RANSAC to find the best transformation matrix
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    
    if mask is None:
        return 0.0
    
    # Calculate the number of inliers
    num_inliers = np.sum(mask)
    
    # Compute the similarity score based on the number of inliers
    similarity_score = num_inliers / len(good_matches)
    return similarity_score


def compute_similarities_testsets(munich_testset, nk_testset, 
                                  munich_path="munich_testset", 
                                  nk_path="nk_testset"):
    """
    This function takes four arguments: 
    - munich_testset, which contains grayscaled images from the Munich database.
    - nk_testset, which contains grayscaled images from the NK collection API.
    - munich path, the path to the directory of the Munich images. 
    - nk_path, the path to the directory of the NK images. 
    
    It then computes the similarity using ORB features.
    It then saves the similarity and the two images as key-value pairs in a dictionary.
    """
    
    similarities = {}
    for nk_img in nk_testset:
        nk_img_path = os.path.join(nk_path, nk_img)
        for munich_img in munich_testset:
            munich_img_path = os.path.join(munich_path, munich_img)
            try:
                similarity = compute_orb_similarity(nk_img_path, munich_img_path)
                similarities[(nk_img, munich_img)] = similarity
            except ValueError as e:
                print(e)
        
    return similarities

# Example usage:
nk_path = "nk_testset"
munich_path = "munich_testset"

if not os.path.exists(nk_path):
    raise FileNotFoundError(f"The path {nk_path} does not exist.")
if not os.path.exists(munich_path):
    raise FileNotFoundError(f"The path {munich_path} does not exist.")

nk_testset = os.listdir(nk_path)
munich_testset = os.listdir(munich_path)

sims = compute_similarities_testsets(munich_testset, nk_testset, munich_path, nk_path)
print(sims)

get_table(sims)

{('dressoir_nk.jpg', 'dressoir_mccp.jpg'): 0.0, ('dressoir_nk.jpg', 'kast_mccp.jpg'): 0.0, ('dressoir_nk.jpg', 'speeltafel_mccp.png'): 0.0, ('dressoir_nk.jpg', 'stoel_mccp.jpg'): 0.0, ('dressoir_nk.jpg', 'tafel_mccp.jpg'): 0.0, ('kast_nk.jpg', 'dressoir_mccp.jpg'): 0.0, ('kast_nk.jpg', 'kast_mccp.jpg'): 0.0, ('kast_nk.jpg', 'speeltafel_mccp.png'): 0.0, ('kast_nk.jpg', 'stoel_mccp.jpg'): 0.0, ('kast_nk.jpg', 'tafel_mccp.jpg'): 0.7142857142857143, ('speeltafel_nk.png', 'dressoir_mccp.jpg'): 0.0, ('speeltafel_nk.png', 'kast_mccp.jpg'): 0.0, ('speeltafel_nk.png', 'speeltafel_mccp.png'): 0.0, ('speeltafel_nk.png', 'stoel_mccp.jpg'): 0.0, ('speeltafel_nk.png', 'tafel_mccp.jpg'): 0.0, ('stoel_nk.jpg', 'dressoir_mccp.jpg'): 0.0, ('stoel_nk.jpg', 'kast_mccp.jpg'): 0.0, ('stoel_nk.jpg', 'speeltafel_mccp.png'): 0.0, ('stoel_nk.jpg', 'stoel_mccp.jpg'): 0.0, ('stoel_nk.jpg', 'tafel_mccp.jpg'): 0.0, ('tafel_nk.jpg', 'dressoir_mccp.jpg'): 0.0, ('tafel_nk.jpg', 'kast_mccp.jpg'): 0.0, ('tafel_nk.jpg', 

,dressoir_mccp,kast_mccp,speeltafel_mccp,stoel_mccp,tafel_mccp
dressoir_nk,0.0,0.0,0.0,0.0,0.000000
kast_nk,0.0,0.0,0.0,0.0,0.714286
speeltafel_nk,0.0,0.0,0.0,0.0,0.000000
stoel_nk,0.0,0.0,0.0,0.0,0.000000
tafel_nk,0.0,0.0,0.0,0.0,0.000000


In [14]:


def extract_orb_features(image_path):
    # Load image
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError(f"Image at path {image_path} could not be loaded.")
    
    # Initialize ORB detector
    orb = cv2.ORB_create()
    
    # Find keypoints and descriptors
    kp, des = orb.detectAndCompute(img, None)
    return kp, des

def match_descriptors(des1, des2):
    matches = []
    for i, d1 in enumerate(des1):
        distances = np.linalg.norm(des2 - d1, axis=1)
        min_idx = np.argmin(distances)
        matches.append((i, min_idx, distances[min_idx]))
    return matches

def ratio_test(matches, ratio_threshold=0.75):
    good_matches = []
    for i, (query_idx, train_idx, distance) in enumerate(matches):
        if i < len(matches) - 1:
            next_distance = matches[i + 1][2]
            if distance < ratio_threshold * next_distance:
                good_matches.append((query_idx, train_idx))
    return good_matches

def compute_orb_similarity(image_path1, image_path2):
    # Extract ORB features
    kp1, des1 = extract_orb_features(image_path1)
    kp2, des2 = extract_orb_features(image_path2)
    
    if des1 is None or des2 is None:
        return 0.0

    # Match descriptors manually
    matches = match_descriptors(des1, des2)
    num_matches = len(matches)
    # Apply ratio test
    good_matches = ratio_test(matches)
    
    if len(good_matches) < 4:
        return 0.0

    src_pts = np.float32([kp1[i].pt for i, _ in good_matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[i].pt for _, i in good_matches]).reshape(-1, 1, 2)

    # Use RANSAC to find the best transformation matrix
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    
    if mask is None:
        return 0.0
    
    # Calculate the number of inliers
    num_inliers = np.sum(mask)
    
    # Compute the similarity score based on the number of inliers
    similarity_score = num_inliers / len(good_matches)
    return similarity_score

def compute_similarities_testsets(munich_testset, nk_testset, 
                                  munich_path="munich_testset", 
                                  nk_path="nk_testset"):
    """
    This function takes four arguments: 
    - munich_testset, which contains grayscaled images from the Munich database.
    - nk_testset, which contains grayscaled images from the NK collection API.
    - munich path, the path to the directory of the Munich images. 
    - nk_path, the path to the directory of the NK images. 
    
    It then computes the similarity using ORB features.
    It then saves the similarity and the two images as key-value pairs in a dictionary.
    """
    
    similarities = {}
    for nk_img in nk_testset:
        nk_img_path = os.path.join(nk_path, nk_img)
        for munich_img in munich_testset:
            munich_img_path = os.path.join(munich_path, munich_img)
            try:
                similarity = compute_orb_similarity(nk_img_path, munich_img_path)
                similarities[(nk_img, munich_img)] = similarity
            except ValueError as e:
                print(e)
        
    return similarities

# Example usage:
nk_path = "nk_testset"
munich_path = "munich_testset"

if not os.path.exists(nk_path):
    raise FileNotFoundError(f"The path {nk_path} does not exist.")
if not os.path.exists(munich_path):
    raise FileNotFoundError(f"The path {munich_path} does not exist.")

nk_testset = os.listdir(nk_path)
munich_testset = os.listdir(munich_path)

sims = compute_similarities_testsets(munich_testset, nk_testset, munich_path, nk_path)
print(sims)

get_table(sims)

{('dressoir_nk.jpg', 'dressoir_mccp.jpg'): 0.0, ('dressoir_nk.jpg', 'kast_mccp.jpg'): 0.0, ('dressoir_nk.jpg', 'speeltafel_mccp.png'): 0.0, ('dressoir_nk.jpg', 'stoel_mccp.jpg'): 0.0, ('dressoir_nk.jpg', 'tafel_mccp.jpg'): 0.0, ('kast_nk.jpg', 'dressoir_mccp.jpg'): 0.0, ('kast_nk.jpg', 'kast_mccp.jpg'): 0.0, ('kast_nk.jpg', 'speeltafel_mccp.png'): 0.0, ('kast_nk.jpg', 'stoel_mccp.jpg'): 0.0, ('kast_nk.jpg', 'tafel_mccp.jpg'): 0.7142857142857143, ('speeltafel_nk.png', 'dressoir_mccp.jpg'): 0.0, ('speeltafel_nk.png', 'kast_mccp.jpg'): 0.0, ('speeltafel_nk.png', 'speeltafel_mccp.png'): 0.0, ('speeltafel_nk.png', 'stoel_mccp.jpg'): 0.0, ('speeltafel_nk.png', 'tafel_mccp.jpg'): 0.0, ('stoel_nk.jpg', 'dressoir_mccp.jpg'): 0.0, ('stoel_nk.jpg', 'kast_mccp.jpg'): 0.0, ('stoel_nk.jpg', 'speeltafel_mccp.png'): 0.0, ('stoel_nk.jpg', 'stoel_mccp.jpg'): 0.0, ('stoel_nk.jpg', 'tafel_mccp.jpg'): 0.0, ('tafel_nk.jpg', 'dressoir_mccp.jpg'): 0.0, ('tafel_nk.jpg', 'kast_mccp.jpg'): 0.0, ('tafel_nk.jpg', 

,dressoir_mccp,kast_mccp,speeltafel_mccp,stoel_mccp,tafel_mccp
dressoir_nk,0.0,0.0,0.0,0.0,0.000000
kast_nk,0.0,0.0,0.0,0.0,0.714286
speeltafel_nk,0.0,0.0,0.0,0.0,0.000000
stoel_nk,0.0,0.0,0.0,0.0,0.000000
tafel_nk,0.0,0.0,0.0,0.0,0.000000
